## Mood classfication using CNN (HAPPY / SAD)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
#image data generator is the package to lable the images & it will automatically lable all the images

In [ ]:
img = image.load_img('/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/training/happy/3.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
i1 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/training/happy/3.jpg')
i1
# 3 dimension metrics are created for the image
# the value ranges from 0-255

In [ ]:
i1.shape
# shape of your image height, weight, rgb

In [ ]:
train = ImageDataGenerator(rescale = 1/200)
validataion = ImageDataGenerator(rescale = 1/200)

# to scale all the images i need to divide with 255
# we need to resize the image using 200, 200 pixel

In [ ]:
train_dataset = train.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/training',
                                         target_size = (200,200),
                                         batch_size = 32,
                                         class_mode = 'binary')
validataion_dataset = validataion.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/validation',
                                          target_size = (200,200),
                                          batch_size = 32,
                                          class_mode = 'binary')

In [ ]:
train_dataset.class_indices

In [ ]:
train_dataset.classes

In [ ]:
# now we are applying maxpooling

model = tf.keras.models.Sequential([ tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',input_shape = (200,200,3)),
                                    tf.keras.layers.MaxPool2D(2,2), #3 filtr we applied hear
                                    #
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    #
                                    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Flatten(),
                                    ##
                                    tf.keras.layers.Dense(512, activation = 'relu'),
                                    #
                                    tf.keras.layers.Dense(1,activation= 'sigmoid')
                                    ]
                                    )

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
              metrics = ['accuracy']
              )

In [ ]:
model_fit = model.fit(train_dataset,epochs = 15)

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/testing'
for i in os.listdir(dir_path ):
    print(i)
    #img = image.load_img(dir_path+ '//'+i, target_size = (200,200))
   # plt.imshow(img)
   # plt.show()

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/testing'

for i in os.listdir(dir_path ):
    img = image.load_img(dir_path+ '//'+i, target_size = (200,200))
    plt.imshow(img)
    plt.show()

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/19. CNN - Happy  or Sad/testing'

for i in os.listdir(dir_path ):
    img = image.load_img(dir_path+ '//'+i, target_size = (200,200))
    plt.imshow(img)
    plt.show()

    x= image.img_to_array(img)
    x=np.expand_dims(x,axis = 0)
    images = np.vstack([x])

    val = model.predict(images)
    if val == 0:
        print( 'i am happy')
    else:
        print('i am not happy')

In [ ]:
import gradio as gr
from PIL import Image
import numpy as np

def predict_mood(image):
    # Resize the image to (200, 200) as expected by the model
    img = image.resize((200, 200))
    # Convert the image to a numpy array
    x = np.array(img)
    # Expand dimensions to create a batch of 1 image
    x = np.expand_dims(x, axis=0)
    # Normalize the image (if the model was trained with normalized inputs)
    # The model was trained with rescale=1/200, so we should apply the same scaling here.
    x = x / 200.0

    # Make prediction
    val = model.predict(x)[0][0]

    # Interpret the prediction
    if val < 0.5:
        return 'Happy'
    else:
        return 'Not Happy'

In [ ]:
iface = gr.Interface(fn=predict_mood,
                     inputs=gr.Image(type="pil", label="Upload an image"),
                     outputs=gr.Text(label="Predicted Mood"),
                     title="Mood Classification (Happy/Not Happy)",
                     description="Upload an image to classify if the person is happy or not happy.")

iface.launch()

In [ ]:
!nvidia-smi